In [3]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import re
import numpy as np
from ast import literal_eval


/home/caumel/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
path = "./jigsaw-toxic-comment-train-processed-seqlen128.csv"
df = pd.read_csv(path)

In [5]:
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,input_word_ids,input_mask,all_segment_id
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"(101, 27746, 31609, 11809, 24781, 10105, 70971...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"(101, 141, 112, 56237, 10874, 106, 10357, 1825...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"(101, 35936, 10817, 117, 146, 112, 181, 30181,...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"(101, 107, 15946, 146, 10944, 112, 188, 13086,...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"(101, 11065, 117, 52523, 117, 10301, 15127, 51...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...
223544,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,0,0,0,0,0,"(101, 131, 43325, 117, 146, 12888, 13028, 1479...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
223545,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,0,0,0,0,0,"(101, 134, 134, 22716, 19111, 101695, 134, 134...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
223546,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,0,0,0,0,0,"(101, 134, 134, 48201, 18969, 10135, 13028, 10...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
223547,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,0,1,0,1,0,"(101, 47430, 11369, 144, 72918, 10731, 30118, ...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [17]:
df.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate', 'input_word_ids', 'input_mask',
       'all_segment_id'],
      dtype='object')

- 'id', 
- 'comment_text', # Text
- 'toxic', # Si es toxico o no
- 'severe_toxic', ?
- 'obscene', ?
- 'threat', ?
- 'insult', ?
- 'identity_hate', ?
- 'input_word_ids',  # Tokens
- 'input_mask', # Si es importante o no
- 'all_segment_id' # Si cuenta con mas de una frase ?

In [6]:
df.iloc[0,:]["input_mask"]

'(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)'

In [7]:
df.at[0, 'input_word_ids']

'(101, 27746, 31609, 11809, 24781, 10105, 70971, 10107, 11019, 10571, 15127, 29115, 23920, 72832, 56684, 30126, 10309, 86095, 46949, 136, 11696, 10309, 10115, 112, 188, 109995, 33269, 12387, 117, 12820, 69177, 10135, 11152, 74212, 10107, 10662, 146, 34584, 10160, 10287, 10482, 80332, 20794, 10858, 119, 12689, 20648, 42658, 10112, 16938, 112, 188, 51600, 10105, 101986, 23953, 10188, 10105, 31311, 15975, 11764, 146, 112, 181, 18675, 11858, 119, 12642, 119, 20862, 119, 11171, 119, 10365, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)'

In [ ]:
# Preprocessing

# Entreno ?
# Fine-tune

# Uso el default

In [8]:
def preprocessing(text):
    return re.sub(r"[.,!?-]", '', text.lower()).replace("\n"," ")

def string_to_tensor(s):
    return torch.tensor(literal_eval(s), dtype=torch.long)

df['input_word_ids'] = df['input_word_ids'].apply(string_to_tensor)
df['input_mask'] = df['input_mask'].apply(string_to_tensor)
df['all_segment_id'] = df['all_segment_id'].apply(string_to_tensor)
# df['comment_text'] = df['comment_text'].apply(preprocessing)
# df['comment_text'] = df['comment_text'].astype(str)

In [9]:
max_length = 512 

#Controlo el maximo tamaño del input ya que BERT solo permite 512

def truncate_or_pad(tensor, max_length):
    # Truncar si el tensor es más largo que max_length
    if len(tensor) > max_length:
        return tensor[:max_length]
    # Rellenar con ceros si el tensor es más corto que max_length
    elif len(tensor) < max_length:
        pad_length = max_length - len(tensor)
        return torch.cat((tensor, torch.zeros(pad_length, dtype=torch.long)))

    return tensor

df['input_word_ids'] = df['input_word_ids'].apply(lambda x: truncate_or_pad(x, max_length))
df['input_mask'] = df['input_mask'].apply(lambda x: truncate_or_pad(x, max_length))
df['all_segment_id'] = df['all_segment_id'].apply(lambda x: truncate_or_pad(x, max_length))


In [10]:
# Controlo el vocabulario de BERT, para saber si estan entre los indices del modelo, para que no exista palabra que el modelo no entienda.

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
vocab_size = len(tokenizer)

print("Tamaño del vocabulario:", vocab_size)

def adjust_token_indices(tensor, vocab_size):
    adjusted_tensor = torch.clamp(tensor, min=0, max=vocab_size - 1)
    return adjusted_tensor

df['input_word_ids'] = df['input_word_ids'].apply(adjust_token_indices, vocab_size=vocab_size)

Tamaño del vocabulario: 30522


In [11]:
# Cargar el modelo preentrenado
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model.eval()

def predict_toxicity_individual(df, model):
    model.to('cpu')  # Asegura que el modelo se ejecute en la CPU
    toxicity_scores = []
    
    for i in range(len(df)):
        # Extrae cada fila como un tensor individual
        input_ids = df['input_word_ids'].iloc[i].unsqueeze(0)  # Añade una dimensión de lote
        attention_mask = df['input_mask'].iloc[i].unsqueeze(0)  # Añade una dimensión de lote
        token_type_ids = df['all_segment_id'].iloc[i].unsqueeze(0)  # Añade una dimensión de lote
        
        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=1)
            score = probabilities[:, 1].item()  # Obtiene la probabilidad de la clase 1 como un número
            toxicity_scores.append(score)
    
    return toxicity_scores

# Aplicar la función predict_toxicity al DataFrame completo
df_input = df.iloc[-10:]

result = predict_toxicity_individual(df_input, model)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
df_input['toxicity_score'] = result
print(df_input[['comment_text', 'toxic', 'toxicity_score']])

# df['toxicity_score'] = result
# print(df[['comment_text', 'toxicity_score']].head())

                                             comment_text  toxic  \
223539  Balancing the two approaches to psychiatry ( b...      0   
223540  == Your name mentioned == \n Hi, I just though...      0   
223541  I've just discovered yet another list: List of...      0   
223542  ::Consensus for ruining Wikipedia? I think tha...      0   
223543  shut down the mexican border withought looking...      0   
223544  :Jerome, I see you never got around to this…! ...      0   
223545  ==Lucky bastard== \n http://wikimediafoundatio...      0   
223546  ==shame on you all!!!== \n\n You want to speak...      0   
223547  MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...      1   
223548  " \n\n == Unicorn lair discovery == \n\n Suppo...      0   

        toxicity_score  
223539        0.504209  
223540        0.491924  
223541        0.513684  
223542        0.494272  
223543        0.493918  
223544        0.517015  
223545        0.510337  
223546        0.506898  
223547        0.534275  
2

/tmp/ipykernel_21282/1083717648.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input['toxicity_score'] = result


In [ ]:
texts = df['comment_text'].tolist()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
inputs = tokenizer("", padding=True, truncation=True, max_length=512, return_tensors="pt")
